In [ ]:
import json
import csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Dense
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report, f1_score
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
with open('json_data_train.json', 'r') as f:
  data = json.load(f)
f.close()
text = []
keywordsInValue = []
keywords = []
keywordType = []
maxLength = 0
maxKeywords = 0
for key in data.keys():
  if len(data[key]['text'].split()) > maxLength:
    maxLength = len(data[key]['text'].split())
  keywordsAndType = data[key]['keywords']
  keywords = list(keywordsAndType.keys())
  for word in keywords:
    wordArray = word.split()
    if len(wordArray) > 1:
      for splitWord in wordArray:
        keywordsAndType[splitWord] = keywordsAndType[word]
  keywords = list(keywordsAndType.keys())
  tokenizer = Tokenizer(keywords)
  tokenizer.fit_on_texts([data[key]['text']])
  tokens = [i for i in tokenizer.word_index.keys()]
  text.append(data[key]['text'])
  encodedKeywords = []
  for word in tokens:
    if word in keywords:
      typeOfKeyword = keywordsAndType[word][0][2]
      if typeOfKeyword == "Process":
        encodedKeywords.append(2)
      elif typeOfKeyword == "Material":
        encodedKeywords.append(3)
      elif typeOfKeyword == "Task":
        encodedKeywords.append(1)
    else:
      encodedKeywords.append(0)
  keywordsInValue.append(encodedKeywords)


In [ ]:
m = 0
maxLength = 0
text = []
keywordsInValue = []
with open("wikidata.txt",'r') as f:
  data = csv.reader(f)
  for row in data:
    if m != 0:
      text.append(row[1])
      if len(row[1]) > maxLength:
        maxLength = len(row[1])
      keywordsInValue.append([int(i) for i in row[2][1:-1].split(", ")])
    else:
      m += 1
print(keywordsInValue)

[[2, 2, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
with open('/content/drive/My Drive/cs4248/train_augmented_annotations.npy', 'rb') as f:
    annotation = np.load(f, allow_pickle=True)
with open('/content/drive/My Drive/cs4248/train_augmented_sentences.npy', 'rb') as f:
    text = np.load(f,allow_pickle=True)
maxLength  = 0
for doc in text:
  if len(doc) > maxLength:
    maxLength = len(doc)
tag_to_ix = {"Task": 0, "Process": 1, "Material": 2, "O": 3}
keywordsInValue = [[tag_to_ix[w] for w in s] for s in annotation]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(text)
X = pad_sequences(X, padding = "post", truncating = "post", maxlen = maxLength)
y = pad_sequences(keywordsInValue, padding = "post", truncating = "post", maxlen = maxLength)
y = [to_categorical(i,num_classes = 4) for i in y]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
print(X_train[0])

[  30 6274 1051 7494   89  299 5675   16 2030    6    1 4325  839 4154
 3054    1 7495    5 7496 7497   15  181 7498  195    4  299 5675 7499
 2498    1 6275 4013    3  175   30 5368    5 5006  493  299 5675  707
    8 2214 2426  427    1  660 1420 2205  190   16  155    3 7500    4
 7501  393 2604  427   19  190    2    6    1  552    3 5004 3620  427
    1  660 1420 2205  190 1280  145    1 6275 1067   16  421  114    6
 7187  370   13  732   12    3    1 7188    4 1031 2604   17 6488   28
 2214 2604    9 3473   18    1 7189   48 7190    5 7191    3    1 5004
 2604 1829    1 7192 7193    7 4385    8 6489  289    2    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
embed = {}
f = open('drive/MyDrive/glove.6B.100d.txt','r')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype = "float32")
	embed [word] = coefs
f.close()

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embed.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
print(X_train.shape)
print(np.array(y_train).shape)
keywordModel = Sequential()
keywordModel.add(Embedding(len(word_index) + 1, 100))
keywordModel.add(Bidirectional(LSTM(64, return_sequences = True,),merge_mode='concat'))
keywordModel.add(TimeDistributed(Dense(4, activation = "softmax")))
keywordModel.compile(loss="categorical_crossentropy", optimizer = "adam")
keywordModel.fit(X_train, np.array(y_train), batch_size = 32, epochs = 7, validation_split = 0.1)

(6729, 297)
(6729, 297, 4)
Epoch 1/7
190/190 [==============================] - 83s 418ms/step - loss: 0.2872 - val_loss: 0.1719
Epoch 2/7
190/190 [==============================] - 78s 411ms/step - loss: 0.1061 - val_loss: 0.0630
Epoch 3/7
190/190 [==============================] - 81s 426ms/step - loss: 0.0399 - val_loss: 0.0272
Epoch 4/7
190/190 [==============================] - 77s 408ms/step - loss: 0.0182 - val_loss: 0.0153
Epoch 5/7
190/190 [==============================] - 78s 413ms/step - loss: 0.0094 - val_loss: 0.0088
Epoch 6/7
190/190 [==============================] - 78s 412ms/step - loss: 0.0056 - val_loss: 0.0051
Epoch 7/7
190/190 [==============================] - 77s 406ms/step - loss: 0.0034 - val_loss: 0.0040


In [ ]:
with open('/content/drive/My Drive/cs4248/test_tokens_annotations.npy', 'rb') as f:
    annotation = np.load(f, allow_pickle=True)
with open('/content/drive/My Drive/cs4248/test_tokens_sentences.npy', 'rb') as f:
    text = np.load(f,allow_pickle=True)
tag_to_ix = {"Task": 0, "Process": 1, "Material": 2, "O": 3}
keywordsInValue = [[tag_to_ix[w] for w in s] for s in annotation]
XTest = tokenizer.texts_to_sequences(text)
XTest = pad_sequences(XTest, padding = "post", truncating = "post", maxlen = maxLength)
yTest = pad_sequences(keywordsInValue, padding = "post", truncating = "post", maxlen = maxLength)
yTest = [to_categorical(i,num_classes = 4) for i in yTest]

In [ ]:
a = keywordModel.predict(XTest)
a = np.argmax(a, axis = -1)
flattened_actual = (np.argmax(np.array(yTest), axis = -1)).flatten()
flattened_output = a.flatten()
print(classification_report(flattened_actual, flattened_output))
print(f1_score(y_pred=flattened_actual, y_true=flattened_output,average = "macro"))

              precision    recall  f1-score   support

           0       0.75      0.93      0.83     10332
           1       0.16      0.12      0.14      2270
           2       0.11      0.06      0.08      1730
           3       0.78      0.72      0.75     15368

    accuracy                           0.71     29700
   macro avg       0.45      0.46      0.45     29700
weighted avg       0.68      0.71      0.69     29700

0.4485252519479535
